In [ ]:
pip install lifelines

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from lifelines import KaplanMeierFitter

# Import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
###
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report,accuracy_score,recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The Dataset

In [ ]:
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
g_H = df.loc[df['ejection_fraction'] > 55]
b_H = df.loc[df['ejection_fraction'] < 55]
f_d = df.loc[df['sex'] == 0]
m_d = df.loc[df['sex'] == 1]

cnd_d = df.loc[df['diabetes'] == 0]
cd_d = df.loc[df['diabetes'] == 1]

ns_d = df.loc[df['smoking'] == 0]
s_d = df.loc[df['smoking'] == 1]

cnbp_d = df.loc[df['high_blood_pressure'] == 1]
cbp_d = df.loc[df['high_blood_pressure'] == 0]

# Exploratory Data Analysis

In [ ]:
g_H['DEATH_EVENT'].value_counts().plot(kind = 'bar', title = 'Good Ejection fraction', xlabel = 'Death', legend = 'Death_event')
plt.ylabel('Count')
plt.legend({'0: Alive, 1: Dead'})
plt.show()
b_H['DEATH_EVENT'].value_counts().plot(kind = 'bar', title = 'Bad Ejection fraction', xlabel = 'Death')
plt.ylabel('Count')
plt.legend({'0: Alive, 1: Dead'})
plt.show()

In [ ]:
sns.boxplot(x = 'sex', y ='ejection_fraction', data = df)
plt.legend({'0: Female', '1: Male'})
plt.ylabel("Ejection Fraction")
plt.show()

In [ ]:
plt.hist(df.ejection_fraction)
plt.title('Ejection Fraction of Patients')
plt.xlabel('Ejection Fraction')
plt.ylabel('count')
plt.show()

sns.distplot(m_d.ejection_fraction)
plt.title('Ejection Fraction of Males')
plt.show()
sns.distplot(f_d.ejection_fraction)
plt.title('Ejection Fraction of Females')
plt.show()
sns.distplot(ns_d.ejection_fraction)
plt.title('Ejection Fraction of Non-Smokers')
plt.show()
sns.distplot(s_d.ejection_fraction)
plt.title('Ejection Fraction of Smokers')
plt.show()

**Null Hypothesis** - There is no difference in Ejection Fraction between Genders. 

**Alternative Hypothesis** - There is a difference in Ejection Fraction between Genders. 

In [ ]:
import scipy.stats as ss 
t_stat, p_val= ss.ttest_ind(m_d.ejection_fraction,f_d.ejection_fraction)
print('P-Value of ejection between Genders - ', p_val)

With a P-Value less than .05 we can **reject** the **Null Hypothesis** and **accept** the **Alternative Hypothesis** that there is a difference in EF between Genders.

# Survial Analysis

In [ ]:
kmf = KaplanMeierFitter()
kmf_g = KaplanMeierFitter()
kmf_ba = KaplanMeierFitter()
kmf_f = KaplanMeierFitter()
kmf_m = KaplanMeierFitter()

kmf_nd = KaplanMeierFitter()
kmf_d = KaplanMeierFitter()
kmf_nb = KaplanMeierFitter()
kmf_b = KaplanMeierFitter()

In [ ]:
kmf.fit(df['time'], df['DEATH_EVENT'], label= 'All Patients')
kmf.plot()
plt.title("The Kaplan-Meier Estimate")
plt.xlabel("Days Passed")
plt.ylabel("Survival Probability")
plt.show()

kmf_g.fit(g_H['time'], g_H['DEATH_EVENT'], label = 'Good Ejection Fraction')
kmf_ba.fit(b_H['time'], b_H['DEATH_EVENT'], label = 'Bad Ejection Fraction')
kmf_g.plot()
kmf_ba.plot()
plt.title("Ejection Fraction - KMF")
plt.xlabel("Days Passed")
plt.ylabel("Survival Probability")
plt.show()

kmf_g.fit(f_d['time'], f_d['DEATH_EVENT'], label = 'Female')
kmf_b.fit(m_d['time'], m_d['DEATH_EVENT'], label = 'Male')
kmf_g.plot()
kmf_b.plot()
plt.title("Sex - KMF")
plt.xlabel("Days Passed")
plt.ylabel("Survival Probability")
plt.show()


kmf_nd.fit(cnd_d['time'],cnd_d['DEATH_EVENT'], label = 'No Diabetes')
kmf_d.fit(cd_d['time'], cd_d['DEATH_EVENT'], label = 'Diabetes')
kmf_nd.plot()
kmf_d.plot()
plt.title("Diabetes - KMF")
plt.xlabel("Days Passed")
plt.ylabel("Survival Probability")
plt.show()

kmf_nb.fit(cnbp_d['time'], cnbp_d['DEATH_EVENT'], label = 'No BP')
kmf_b.fit(cbp_d['time'], cbp_d['DEATH_EVENT'], label = 'BP')
kmf_nb.plot()
kmf_b.plot()
plt.title("BP - KMF")
plt.xlabel("Days Passed")
plt.ylabel("Survival Probability")
plt.show()

# Models

In [ ]:
df2 = df
y = df2['DEATH_EVENT'].values
X = df2.drop('DEATH_EVENT', axis= 1).values


# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify= y)

# Supervised Models

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)

# Fit the classifier to the training data
knn.fit(X_train, y_train)

# Predict the labels of the test data: y_pred
y_pred = knn.predict(X_test)

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Evaluate test-set roc_auc_score
KNN_6_Classifier_roc = roc_auc_score(y_test, y_pred)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(KNN_6_Classifier_roc))

fpr, tpr, _ = roc_curve(y_test,  y_pred)
auc2 = roc_auc_score(y_test, y_pred)
plt.plot(fpr,tpr,label="KNN-6-Classifier, auc="+str(auc2))
plt.legend(loc=4)
plt.show()

In [ ]:
# Create the classifier: logreg
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = logreg.predict(X_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

LogReg_ROC = roc_auc_score(y_test, y_pred)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(LogReg_ROC))

fpr, tpr, _ = roc_curve(y_test,  y_pred)
auc2 = roc_auc_score(y_test, y_pred)
plt.plot(fpr,tpr,label="LogisticRegression, auc="+str(auc2))
plt.legend(loc=4)
plt.show()

In [ ]:
# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}

# Instantiate the logistic regression classifier: logreg
logreg = LogisticRegression()

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, param_grid=param_grid, cv = 5)

# Fit it to the training data
logreg_cv.fit(X_train, y_train)

In [ ]:
y_pred = logreg_cv.predict(X_test)
# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))

LogReg_CV = roc_auc_score(y_test, y_pred)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(LogReg_CV))

fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr,label="LogisticRegression_CV, auc="+str(LogReg_CV))
plt.legend(loc=4)
plt.show()

In [ ]:
# Setup the pipeline steps: steps
steps = [('scaler', StandardScaler()),
        ('logreg', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the pipeline to the training set: knn_scaled
knn_scaled = pipeline.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = knn_scaled.predict(X_test)

# Instantiate and fit a k-NN classifier to the unscaled data
knn_unscaled = KNeighborsClassifier().fit(X_train, y_train)

# Compute and print metrics
print('Accuracy with Scaling: {}'.format(knn_scaled.score(X_test, y_test)))
print('Accuracy without Scaling: {}'.format(knn_unscaled.score(X_test, y_test)))

LogReg_Scaled = roc_auc_score(y_test, y_pred)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(LogReg_Scaled))

fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr,label="LogisticRegression_Scaled, auc="+str(LogReg_Scaled))
plt.legend(loc=4)
plt.show()

In [ ]:
# Instantiate dt
dt = DecisionTreeClassifier(max_depth=2, random_state=1)

# Instantiate ada
ada_dt = AdaBoostClassifier(base_estimator=dt, n_estimators=180, random_state=1)

# Fit ada to the training set
ada_dt.fit(X_train,y_train)

# Compute the probabilities of obtaining the positive class
y_pred_prob_dt = ada_dt.predict_proba(X_test)[:,1]

# Evaluate test-set roc_auc_score
ada_dt_roc_auc = roc_auc_score(y_test, y_pred_prob_dt)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(ada_dt_roc_auc))

fpr, tpr, _ = roc_curve(y_test,  y_pred_prob_dt)
auc1 = roc_auc_score(y_test, y_pred_prob_dt)
plt.plot(fpr,tpr,label="Ada_boost_DT, auc="+str(auc1))
plt.legend(loc=4)
plt.show()

In [ ]:
rfc = RandomForestClassifier()
# Instantiate ada
ada_rf = AdaBoostClassifier(base_estimator=rfc, n_estimators=180, random_state=1)

# Fit ada to the training set
ada_rf.fit(X_train,y_train)

# Compute the probabilities of obtaining the positive class
y_pred_prob_rf = ada_rf.predict_proba(X_test)[:,1]

# Evaluate test-set roc_auc_score
ada_rf_roc_auc = roc_auc_score(y_test, y_pred_prob_rf)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(ada_rf_roc_auc))

fpr, tpr, _ = roc_curve(y_test,  y_pred_prob_rf)
auc2 = roc_auc_score(y_test, y_pred_prob_rf)
plt.plot(fpr,tpr,label="Ada_boost_RF, auc="+str(auc2))
plt.legend(loc=4)
plt.show()

# Unsupervised Models

In [ ]:
# Instantiate a DecisionTreeClassifier 'dt' with a maximum depth of 6
dt = DecisionTreeClassifier(max_depth=3)

# Fit dt to the training set

dt.fit(X_train, y_train)

# Predict test set labels
y_pred = dt.predict(X_test)

# Compute test set accuracy  
acc = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(acc))

# Evaluate test-set roc_auc_score
DT_AUC = roc_auc_score(y_test, y_pred)

# Print roc_auc_score
print('ROC AUC score: {:.2f}'.format(DT_AUC))

fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr,label="Ada_boost_RF, auc="+str(DT_AUC))
plt.legend(loc=4)
plt.show()

In [ ]:
from sklearn import tree
df2 = df
y = df2['DEATH_EVENT']
X = df2.drop('DEATH_EVENT', axis= 1)
fig = plt.figure(figsize=(25,20))

_ = tree.plot_tree(dt, 
                   feature_names=X.columns,  
                   class_names='AD',
                   filled=True)

**As Many know a low EF is a potential factor for a Cardiac Event. Looking at the DT above, we see Serum_Creatinine & platelets levels are another possible factor.**